In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt

import math
import os
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
data_path = 'data/'

In [4]:
def replace_labes(label_data):
	    dict_map = {0.0 : 0.0, 1.0 : 1.0, 2.0 : 2.0, 3.0 : 3.0, 7.0 : 4.0, 9.0 : 5.0,
	                19.0 : 6.0, 21.0 : 7.0, 99.0 : 7.0}
	    for i in range(len(label_data)):
	        if label_data[i] in dict_map:
	            label_data[i] = dict_map[label_data[i]]-1
	        else :
	            label_data[i] = 0.0
	    return label_data

def check_unnamed(dataframe):
	if 'Unnamed: 0' in list(dataframe):
		dataframe = dataframe.drop('Unnamed: 0',axis=1,errors='ignore')
	return dataframe

In [5]:
diagnosed_data = pd.read_csv(data_path + '22_COMB_diag_hotData.csv', low_memory=False)
diagnosed_col = pd.read_csv(data_path + '22_COMB_diag_col.csv', low_memory=False)

In [6]:
cold_col_names_list = pd.read_csv(data_path + 'diagnosed_for_col_names.csv', low_memory=False)
cold_col_names_list = check_unnamed(cold_col_names_list)
cold_col_names_list = cold_col_names_list[cold_col_names_list.columns[0]].tolist()

In [7]:
cold_col_names_list[10]

'highest_qualification'

In [14]:
def find_hot_matches(col_index_list_to_remove, hot_col_names_list):
    hot_index_list = list()

    for cold in col_index_list_to_remove:
        for hot in hot_col_names_list:
            if cold in hot:
                index = hot_col_names_list.index(hot)
                if index not in hot_index_list :
                    hot_index_list.append(index)
    return (hot_index_list)

In [15]:
def split_data(diagnosed_data, diagnosed_col, col_index_list_to_remove):
	# Checking if column 'Unnamed : 0' is in the dataframe
	# If present, remove it
	diagnosed_col = check_unnamed(diagnosed_col)
	diagnosed_data = check_unnamed(diagnosed_data)

	hot_col_names_list = list(diagnosed_data)
	to_remove_hot_col = find_hot_matches(col_index_list_to_remove, hot_col_names_list)

	diagnosed_data.drop(diagnosed_data.columns[to_remove_hot_col],axis=1,inplace=True,errors='ignore')
	new_hot_col_list = list(diagnosed_data)

	assert (diagnosed_data.shape[0] == diagnosed_col.shape[0])
	split_train = int(diagnosed_data.shape[0] * 0.60)
	split_test = split_train + int(diagnosed_data.shape[0] * 0.25)

	diagnosed_data = np.array(diagnosed_data.astype(float))
	diagnosed_col = np.array(diagnosed_col.astype(float))[:,0]

	diagnosed_col = replace_labes(diagnosed_col)
	diagnosed_col = to_categorical(diagnosed_col.astype('int32'), nb_classes=None)

	train_data = diagnosed_data[:split_train]
	train_label = diagnosed_col[:split_train]

	test_data = diagnosed_data[split_train:split_test]
	test_label = diagnosed_col[split_train:split_test]

	valid_data = diagnosed_data[split_test:]
	valid_label = diagnosed_col[split_test:]

	print('Features Removed : ', col_index_list_to_remove)
	print('Train dataset : ', train_data.shape, train_label.shape)
	print('Test dataset : ', test_data.shape, test_label.shape)
	print('Validation Dataset : ', valid_data.shape, valid_label.shape)

	return(train_data, train_label, test_data, test_label, valid_data, valid_label)

In [16]:
tr, trl, ts, tsl, v, vl = split_data(diagnosed_data, diagnosed_col, [cold_col_names_list[10]])

Features Removed :  ['highest_qualification']
Train dataset :  (94821, 292) (94821, 7)
Test dataset :  (39509, 292) (39509, 7)
Validation Dataset :  (23706, 292) (23706, 7)


In [17]:
len(cold_col_names_list)

58